In [1]:
!pip install dask[dataframe]
!pip install dask[diagnostics]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 5.0 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nissshiii","key":"f8d0ae2ba9749aa34ef58896924a4424"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d tzelal/binance-bitcoin-dataset-1s-timeframe-p1

Dataset URL: https://www.kaggle.com/datasets/tzelal/binance-bitcoin-dataset-1s-timeframe-p1
License(s): CC0-1.0
 99% 2.66G/2.68G [00:10<00:00, 302MB/s]
100% 2.68G/2.68G [00:10<00:00, 271MB/s]


In [4]:
#!unzip -p binance-bitcoin-dataset-1s-timeframe-p1.zip > extracted_file.csv


In [5]:
import zipfile
import dask.dataframe as dd
import os

# Define the path to the zip file and the CSV file inside it
zip_file_path = '/content/binance-bitcoin-dataset-1s-timeframe-p1.zip'
csv_file_name = 'half1_BTCUSDT_1s.csv'  # The name of your CSV file
#temp_csv_path = '/content/sample_data/temp'  # Temporary path for extraction

# Extract the CSV file from the zip archive
with zipfile.ZipFile(zip_file_path) as z:
    z.extract(csv_file_name, path='/content/')  # Extract to /content/ directory

# Load the extracted CSV file into a Dask DataFrame
dask_df = dd.read_csv('/content/half1_BTCUSDT_1s.csv')

# Show information about the DataFrame
print(dask_df.info())

# Optionally, clean up the temporary CSV file
#os.remove(temp_csv_path)  # Remove the temporary file after loading


<class 'dask_expr.DataFrame'>
Columns: 12 entries, Open Time to Ignore
dtypes: float64(8), int64(2), string(2)None


In [6]:
#!unzip binance-bitcoin-dataset-1s-timeframe-p1.zip

In [7]:
import dask.dataframe as dd
file_path = '/content/half1_BTCUSDT_1s.csv'
dask_df = dd.read_csv(file_path)
print(dask_df.info())

<class 'dask_expr.DataFrame'>
Columns: 12 entries, Open Time to Ignore
dtypes: float64(8), int64(2), string(2)None


In [8]:
# Clean column names
dask_df.columns = [col.replace(' ', '_').replace('.', '').replace('-', '').replace('(', '').replace(')', '') for col in dask_df.columns]
print("Cleaned column names:")
print(dask_df.columns.tolist())

Cleaned column names:
['Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_Time', 'Quote_Asset_Volume', 'Number_of_Trades', 'Taker_Buy_Base_Asset_Volume', 'Taker_Buy_Quote_Asset_Volume', 'Ignore']


In [9]:
import yaml

# Define the schema with column names and separator
schema = {
    'columns': dask_df.columns.tolist(),
    'separator': '|'
}

# Write the schema to a YAML file
with open('/content/schema.yaml', 'w') as file:
    yaml.dump(schema, file)

print("YAML schema file created.")

YAML schema file created.


In [10]:
# Load the YAML file
with open('/content/schema.yaml') as file:
    loaded_schema = yaml.load(file, Loader=yaml.FullLoader)

# Validate number of columns
if len(dask_df.columns) == len(loaded_schema['columns']):
    print("Column count is valid.")
else:
    print("Column count mismatch.")

# Validate column names
for column in loaded_schema['columns']:
    if column not in dask_df.columns:
        print(f"Column '{column}' is missing in the DataFrame.")
    else:
        print(f"Column '{column}' is present in the DataFrame.")

Column count is valid.
Column 'Open_Time' is present in the DataFrame.
Column 'Open' is present in the DataFrame.
Column 'High' is present in the DataFrame.
Column 'Low' is present in the DataFrame.
Column 'Close' is present in the DataFrame.
Column 'Volume' is present in the DataFrame.
Column 'Close_Time' is present in the DataFrame.
Column 'Quote_Asset_Volume' is present in the DataFrame.
Column 'Number_of_Trades' is present in the DataFrame.
Column 'Taker_Buy_Base_Asset_Volume' is present in the DataFrame.
Column 'Taker_Buy_Quote_Asset_Volume' is present in the DataFrame.
Column 'Ignore' is present in the DataFrame.


In [11]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Write the DataFrame to a gzipped pipe-separated text file with progress tracking
output_file_path = '/content/extracted_file.pipe.gz'

with ProgressBar():
    dask_df.to_csv(output_file_path, sep='|', compression='gzip', index=False, single_file=True)

# Summary
total_rows = dask_df.shape[0].compute()  # Total number of rows
total_columns = dask_df.shape[1]          # Total number of columns
file_size = os.path.getsize(output_file_path)  # File size in bytes

# Print summary
print(f"Total number of rows: {total_rows}")
print(f"Total number of columns: {total_columns}")
print(f"File size (in bytes): {file_size}")


[########################################] | 100% Completed | 46m 55s
Total number of rows: 110671732
Total number of columns: 12
File size (in bytes): 2841799376


In [13]:
# Summary
# Use persist() for faster row count computation
total_rows = dask_df.shape[0].persist()  # Cache the result in memory
total_columns = dask_df.shape[1]          # Total number of columns

# Trigger computation to get the actual number of rows
total_rows_value = total_rows.compute()

# File size calculation
file_size = os.path.getsize(output_file_path)  # File size in bytes

# Print summary
print(f"Total number of rows: {total_rows_value}")
print(f"Total number of columns: {total_columns}")
print(f"File size (in bytes): {file_size}")


Total number of rows: 110671732
Total number of columns: 12
File size (in bytes): 2841799376
